1. Generate all permutations of a given string.
2. Generate all unique letter combinations of a string.
3. Return indices which add up to a sum.
4. Search for a number in a sorted, but rotated array.
5. Implement nCk
6. Combination sum -- all possible combinations which lead to the target sum.
7. Combination sum -- single use only, without repeatitions.
8. Palindrome Partitions
9. Skyline problem.
---

# 1. Generate all permutations of a string.

```
abc:

abc
acb

bac
bca

cab
cba
```

If you fix a character `anchor`, then you can find the permutations `p` of the remaining characters, and generate a *complete* permutation as `[anchor] + p`.

This process can be followed recursively. The base case is when your sequence has only 1 character. Then you just return the `[seq]` as the list of all permutations.

In [20]:
class GenPermutations:
    def solve(self, text):
        def permute(seq):
            """
            Generates all permutations for a string.
            """
            if len(seq) == 1:
                return [seq]
            else:
                current = []
                for i in range(len(seq)):
                    anchor = seq[i]
                    remainder = seq[:i] + seq[i+1:]
                    for p in permute(remainder):
                        current.append([anchor] + p)
                        
                return current
            
        return permute(text)

In [19]:
o = GenPermutations()
o.solve(list('abc'))

[['a', 'b', 'c'],
 ['a', 'c', 'b'],
 ['b', 'a', 'c'],
 ['b', 'c', 'a'],
 ['c', 'a', 'b'],
 ['c', 'b', 'a']]

# 2. Generate all unique combinations of a string.

This is also a powerset. And it includes the empty string.

In [51]:
class Combinations:
    def solve(self, text):
        ans = []
        chars = list(text)
        
        def search(start, temp):
            ans.append(''.join(temp))
            if start < len(chars):
                for i in range(start, len(chars)):
                    temp.append(chars[i])
                    search(i+1, temp)
                    temp.pop()
                    
        search(0, [])
        return ans

In [52]:
o = Combinations()
o.solve('abc')

['', 'a', 'ab', 'abc', 'ac', 'b', 'bc', 'c']

# 3. Return indices of an array, which add up to a sum.

```
nums: [2, 10, 7, 5]
target: 9
output: [0, 2]
```

In [40]:
class Indices:
    def solve(self, array, target):
        positions = {}
        for ix, num in enumerate(array):
            diff = target - num
            if diff == 0:
                return [ix]
            else:
                if diff in positions:
                    return [ix, positions[diff]]
                else:
                    positions[num] = ix
                    
        return None

In [42]:
o = Indices()
o.solve([2, 10, 7, 5], 9)

[2, 0]

# 4. Search for a number in a sorted but rotated array.

*Can you do this in `log n` time?*

```
1 2 3 4 5
---------
4 5 1 2 3
```
A rotated array will always result in TWO sorted sub-arrays.

In [43]:
class SearchRotated:
    def solve(self, array, x):
        i = 0
        j = len(array)-1
        
        while i <= j:
            mid = i + (j-i)//2
            if array[mid] == x:
                return True
            else:
                if array[mid] <= array[-1]:
                    # mid to end is sorted.
                    if x > array[mid] and x <= array[-1]:
                        i = mid + 1
                    else:
                        j = mid - 1
                else:
                    # start to mid is sorted.
                    if x >= array[0] and x < array[mid]:
                        j = mid - 1
                    else:
                        i = mid + 1
                        
        return None

In [47]:
o = SearchRotated()
print(o.solve([4, 5, 7, 1, 2, 3], 9))

None


# 5. Implement `n C k`

In [92]:
class NCK:
    def solve(self, n, k):
        """
        n: [1, 2, 3, ..., n]
        Return all possible combinations of k numbers from n.
        
        [1 2 3]
        
        1
        1 2
        1 pop because exceeded
        1 3
        pop
        end of loop
        2 3
        end of loop
        """
        nums = list(range(1, n+1))
        ans = []
        
        def choose(start, t, temp):
            """
            Choose t numbers, starting from position `start`.
            """
            if t == 0:
                ans.append(list(temp))  # Recursion stops here.
            else:
                for i in range(start, len(nums)):
                    temp.append(nums[i])
                    choose(i+1, t-1, temp)
                    temp.pop()
                
        choose(0, k, [])
        return ans

In [93]:
o = NCK()
o.solve(4, 1)

[[1], [2], [3], [4]]

In [94]:
o = NCK()
o.solve(4, 2)

[[1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]]

In [95]:
o = NCK()
o.solve(4, 3)

[[1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]

In [96]:
o = NCK()
o.solve(4, 3)

[[1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]

# 6. Implement combination sum

```
[2, 3, 4], 4

[2, 2]
[4]
```

In [59]:
class ComboSum:
    def solve(self, nums, target):
        """
        Be greedy -- keep adding the current number
        till the diff next number is lower than the diff.
        If you reach 0, great, else pop, and move to the next index.
        """
        ans = []
        
        def search(start, t, path):
            if t == 0:
                ans.append(list(path))
            else:
                for i in range(start, len(nums)):
                    if nums[i] <= t:
                        path.append(nums[i])
                        search(i, t-nums[i], path)
                        path.pop()
                        
        search(0, target, [])
        return ans

In [64]:
o = ComboSum()
o.solve([2, 3, 4, 5, 7], 7)

[[2, 2, 3], [2, 5], [3, 4], [7]]

# 6 alt. Combinations, by subsetting.

```
[1, 2, 3]

[1, 2, 3]
    []
    a = 1, [2, 3]
            []
            a = 2, [3]
            [[2, 3]]
```

In [100]:
class CombAlt:
    def solve(self, nums):
        ans = []
        nums.sort()
        def combos(seq):
            if len(seq) == 1:
                return [seq]  # Only 1 possible combination.
            else:
                current = []
                prev = nums[-1] + 1
                for i in range(len(seq)):
                    anchor = seq[i]
                    if prev == anchor:
                        continue
                    
                    current.append([anchor])
                    rest = seq[i+1:]
                    for c in combos(rest):
                        current.append([anchor] + c)
                        
                    prev = anchor
                        
                return current
            
        return combos(nums)
        return ans

In [101]:
o = CombAlt()
o.solve([1, 2, 2, 4])

[[1],
 [1, 2],
 [1, 2, 2],
 [1, 2, 2, 4],
 [1, 2, 4],
 [1, 4],
 [2],
 [2, 2],
 [2, 2, 4],
 [2, 4],
 [4]]

# 7. Combination Sum -- single use only

The same problem, but you can only use a number once.

In [111]:
class ComboSumSingle:
    def solve(self, nums, target):
        nums.sort()
        ans = []
        
        def search(start, t, path):
            """
            This function, returns the `path` that adds up to `t`.
            """
            if t == 0:
                ans.append(list(path))
            else:
                prev = nums[-1]+1
                for i in range(start, len(nums)):
                    n = nums[i]
                    if prev == n:
                        continue
                    else:
                        prev = n
                    
                    if n <= t:
                        path.append(n)
                        search(i+1, t-n, path)
                        path.pop()
                    else:
                        return
                    
        search(0, target, [])
        return ans

In [113]:
o = ComboSumSingle()
o.solve([10, 1, 2, 7, 6, 1, 5], 8)

[[1, 1, 6], [1, 2, 5], [1, 7], [2, 6]]

# 8. Palindrome Partition

```
aba => [a, b, a], [aba]
aab => [a, a, b], [aa, b]
```

Split a sequence into all possible palindromic sequences.

In [121]:
class PalindromePartition:
    def solve(self, text):
        pal = [[0 for _ in text] for _ in text]
        for i in range(len(text)):
            pal[i][i] = 1
            if i > 0:
                pal[i-1][i] = int(text[i-1] == text[i])
                
        for gram in range(3, len(text)+1):
            for i in range(len(text)-gram+1):
                j = i + gram - 1
                if text[i] == text[j] and pal[i+1][j-1]:
                    pal[i][j] = 1
                    
        for row in pal:
            print(row)
            
        ans = []
        def search(start, temp):
            """
            Given a start position, this function finds all cuts for which the 
            substrings are also palindromes.
            """
            if start == len(text):
                ans.append(list(temp))
            else:
                for end in range(start, len(text)):
                    if pal[start][end]:
                        temp.append((start, end))
                        search(end+1, temp)
                        temp.pop()
                        
        search(0, [])
        return [[text[start:end+1] for start, end in temp] for temp in ans]

In [122]:
o = PalindromePartition()
o.solve('aabaa')

[1, 1, 0, 0, 1]
[0, 1, 0, 1, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 1, 1]
[0, 0, 0, 0, 1]


[['a', 'a', 'b', 'a', 'a'],
 ['a', 'a', 'b', 'aa'],
 ['a', 'aba', 'a'],
 ['aa', 'b', 'a', 'a'],
 ['aa', 'b', 'aa'],
 ['aabaa']]

# 9. Skyline Problem

Given triplets of (start, height, end), find the the skyline.
```
(1, 4, 3), (2, 7, 5), (4, 2, 7), (9, 4, 10), (10, 6, 11)

1,4
2,7
5,2
0,7
```